In [1]:
!pip uninstall -y screenpro2
!pip install git+https://github.com/ArcInstitute/screenpro2.git@dev
# !pip install ScreenPro2

Found existing installation: ScreenPro2 0.4.12
Uninstalling ScreenPro2-0.4.12:
  Successfully uninstalled ScreenPro2-0.4.12
  Cloning https://github.com/ArcInstitute/screenpro2.git (to revision dev) to /tmp/pip-req-build-_kq2dxr3
  Running command git clone --filter=blob:none --quiet https://github.com/ArcInstitute/screenpro2.git /tmp/pip-req-build-_kq2dxr3
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  Branch 'dev' set up to track remote branch 'dev' from 'origin'.
  Resolved https://github.com/ArcInstitute/screenpro2.git to commit eeee5f0c69d908f3ac55e5dbb8935d1aac745ba7
  Preparing metadata (setup.py) ... done
  Created wheel for ScreenPro2: filename=ScreenPro2-0.4.12-py3-none-any.whl size=42978 sha256=5bba5ad748f94c8a848961f694c9caf2c6647ea5131b415e243d7992f2201c8c
  Stored in directory: /tmp/pip-ephem-wheel-cache-rxdu61i_/wheels/4c/5b/3b/b0fb7e5295ecc7453394ca2c5680bf423c37c4917767218de0
Successfully built ScreenPro2


In [2]:
drug_names = {
    'Pi': 'PARPi',
    'Ri': 'ATRi',
    'Wi': 'WEE1i',
    'Mi': 'ATMi',
    'Ki': 'DNAPKi',
    'PiRi': 'PARPi+ATRi',
    'PiWi': 'PARPi+WEE1i',
    'PiMi': 'PARPi+ATMi',
    'PiKi': 'PARPi+DNAPKi'
}

In [3]:
from glob import glob

import numpy as np
import pandas as pd 
import anndata as ad
import scanpy as sc

import screenpro as scp

import matplotlib.pyplot as plt

sc.settings.set_figure_params(
    dpi=150, format='svg', frameon=False, figsize=(3, 3), color_map='RdGy', 
    facecolor='white', 
    vector_friendly=True
)

In [4]:
import datetime

import matplotlib
import matplotlib.ticker as ticker

from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import font_manager as fm
from matplotlib import rcParams, rc_context

from screenpro.plotting._utils import almost_black, dark2


matplotlib.use('cairo')

font_files = fm.findSystemFonts(fontpaths='/home/abea/miniconda3/envs/screenpro2/fonts/', fontext='ttf')

for font_file in font_files:
    fm.fontManager.addfont(font_file)


# {f.name for f in matplotlib.font_manager.fontManager.ttflist}

rcParams['font.sans-serif'] = 'Helvetica'
rcParams['font.family'] = ['Helvetica']
rcParams['figure.dpi'] = 140

rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42

## Step 1: FASTQ processing


In [5]:
library_table = pd.read_csv(
    'CRISPRi_v2_human_librarytable.txt.gz', 
    sep = '\t', index_col=False, low_memory=False)

In [6]:
samples = [
    s.split('/')[-1].replace('.fastq.gz','')
    for s in glob('fastq/A549_CRISPRi*.fastq.gz')
]

samples.sort()

In [7]:
samples

['A549_CRISPRi_DMSO_rep1',
 'A549_CRISPRi_DMSO_rep2',
 'A549_CRISPRi_Ki_rep1',
 'A549_CRISPRi_Ki_rep2',
 'A549_CRISPRi_Mi_rep1',
 'A549_CRISPRi_Mi_rep2',
 'A549_CRISPRi_PiKi_rep1',
 'A549_CRISPRi_PiKi_rep2',
 'A549_CRISPRi_PiMi_rep1',
 'A549_CRISPRi_PiMi_rep2',
 'A549_CRISPRi_PiRi_rep1',
 'A549_CRISPRi_PiRi_rep2',
 'A549_CRISPRi_PiWi_rep1',
 'A549_CRISPRi_PiWi_rep2',
 'A549_CRISPRi_Pi_rep1',
 'A549_CRISPRi_Pi_rep2',
 'A549_CRISPRi_Ri_rep1',
 'A549_CRISPRi_Ri_rep2',
 'A549_CRISPRi_T0_rep1',
 'A549_CRISPRi_T0_rep2',
 'A549_CRISPRi_Wi_rep1',
 'A549_CRISPRi_Wi_rep2']

### run `guideCounter`

In [10]:
counter = scp.GuideCounter(cas_type = 'cas9', library_type = 'single_guide_design')

In [11]:
library_table

,sgId,sublibrary,gene,transcripts,sequence
0,A1BG_-_58858617.23-P1,h3_top5,A1BG,P1,GGAGACCCAGCGCTAACCAG
1,A1BG_-_58858788.23-P1,h3_top5,A1BG,P1,GGGGCACCCAGGAGCGGTAG
2,A1BG_+_58858964.23-P1,h3_top5,A1BG,P1,GCTCCGGGCGACGTGGAGTG
3,A1BG_-_58858630.23-P1,h3_top5,A1BG,P1,GAACCAGGGGTGCCCAAGGG
4,A1BG_+_58858549.23-P1,h3_top5,A1BG,P1,GGCGAGGAACCGCCCAGCAA
...,...,...,...,...,...
209065,non-targeting_03785,h7_supp5,negative_control,na,GCTACTCCGCCCCGCGGGAG
209066,non-targeting_03786,h7_supp5,negative_control,na,GTGGCCGTTCATGGGACCGG
209067,non-targeting_03787,h7_supp5,negative_control,na,GAACTCTGTAGAAGGGACCG
209068,non-targeting_03788,h7_supp5,negative_control,na,GCTCGACAGCGACTGAAGAG


In [12]:
counter.load_library(
    'CRISPRi_v2_human_librarytable.txt.gz', sep = '\t', index_col=False,
    # 'JR_V3lib_top2_v2.csv', sep = ',', 
    verbose = True,
    low_memory=False
)

Trimming protospacer sequences in 'protospacer' column.
Library table successfully loaded.
total # of cas9 sgRNAs: 205648


In [13]:
counter.get_counts_matrix(
    fastq_dir = 'fastq',
    samples = samples,
    # write='force',
    trim_first_g=True,
    verbose = True
)

A549_CRISPRi_DMSO_rep1
count file exists ...
% mapped reads 88.63374797857534
A549_CRISPRi_DMSO_rep2
count file exists ...
% mapped reads 80.83309342989601
A549_CRISPRi_Ki_rep1
count file exists ...
% mapped reads 90.24498424671202
A549_CRISPRi_Ki_rep2
count file exists ...
% mapped reads 89.39757479089117
A549_CRISPRi_Mi_rep1
count file exists ...
% mapped reads 89.78796940558229
A549_CRISPRi_Mi_rep2
count file exists ...
% mapped reads 89.46854195668756
A549_CRISPRi_PiKi_rep1
count file exists ...
% mapped reads 90.54211297399893
A549_CRISPRi_PiKi_rep2
count file exists ...
% mapped reads 89.53799107315274
A549_CRISPRi_PiMi_rep1
count file exists ...
% mapped reads 89.40129422987994
A549_CRISPRi_PiMi_rep2
count file exists ...
% mapped reads 89.56277272210386
A549_CRISPRi_PiRi_rep1
count file exists ...
% mapped reads 88.53951639574869
A549_CRISPRi_PiRi_rep2
count file exists ...
% mapped reads 82.28354294796591
A549_CRISPRi_PiWi_rep1
count file exists ...
% mapped reads 78.658207943

In [14]:
counter.counts_mat.sum() / 10**6

A549_CRISPRi_DMSO_rep1     48.718589
A549_CRISPRi_DMSO_rep2     28.086767
A549_CRISPRi_Ki_rep1       37.695912
A549_CRISPRi_Ki_rep2       23.865067
A549_CRISPRi_Mi_rep1       32.910419
A549_CRISPRi_Mi_rep2       46.517719
A549_CRISPRi_PiKi_rep1     37.236937
A549_CRISPRi_PiKi_rep2     78.800999
A549_CRISPRi_PiMi_rep1     37.714290
A549_CRISPRi_PiMi_rep2    108.729121
A549_CRISPRi_PiRi_rep1     43.599360
A549_CRISPRi_PiRi_rep2     19.454159
A549_CRISPRi_PiWi_rep1     40.225769
A549_CRISPRi_PiWi_rep2     37.929204
A549_CRISPRi_Pi_rep1       45.441083
A549_CRISPRi_Pi_rep2       33.736242
A549_CRISPRi_Ri_rep1       54.166237
A549_CRISPRi_Ri_rep2       32.538540
A549_CRISPRi_T0_rep1       48.168472
A549_CRISPRi_T0_rep2       33.978428
A549_CRISPRi_Wi_rep1       86.448619
A549_CRISPRi_Wi_rep2       34.393649
dtype: float64

In [15]:
adata = counter.build_counts_anndata()

In [110]:
adata.obs['condition'] = adata.obs.index.str.split('_').str[-2]
adata.obs['replicate'] = adata.obs.index.str[-1:].astype(int)
# adata.obs['pop_doublings'] = [1 if cond != 'T0' else 0 for cond in adata.obs['condition']]

adata.var['targetType'] = ['gene' if t != 'negative_control' else 'negative_control' for t in adata.var.target]

In [111]:
adata.write_h5ad('A549_CRISPRi_screens.h5ad.gz', compression='gzip')

### load counts and metadata

In [8]:
adata = ad.read_h5ad('A549_CRISPRi_screens.h5ad.gz')

# adata.obs = meta 
# adata.obs.treatment = adata.obs.treatment.str.replace('-','T0')
# adata.obs = adata.obs.rename(columns={'treatment':'condition','rep':'replicate'})

# adata.obs['pop_doublings'] = meta['pop doublings'].replace('-',np.nan).astype(float)

In [9]:
adata.var['transcript'] = library_table.set_index('sgId').loc[adata.var.index, 'transcripts']
adata.obs['replicate'] = adata.obs['replicate'].astype(int)

___
<!-- - filter low counts -->

In [10]:
adata0 = adata.copy()

___

In [11]:
fig1, axes = plt.subplots(ncols=5,
                         figsize=(10,3), 
                         gridspec_kw={'wspace': 0.3, 'left': 0.12,'top':.8,'bottom':.3}, 
                         # sharey=True
                        )
scp.pl.plotReplicateScatter(axes[0], adata, 'A549_CRISPRi_T0_rep1','A549_CRISPRi_T0_rep2','T0',min_val=0,max_val=4, log_transform=True)
scp.pl.plotReplicateScatter(axes[1], adata, 'A549_CRISPRi_DMSO_rep1','A549_CRISPRi_DMSO_rep2','DMSO',min_val=0,max_val=4, log_transform=True)
scp.pl.plotReplicateScatter(axes[2], adata, 'A549_CRISPRi_Pi_rep1','A549_CRISPRi_Pi_rep2','Pi',min_val=0,max_val=4, log_transform=True)
scp.pl.plotReplicateScatter(axes[3], adata, 'A549_CRISPRi_Pi_rep1','A549_CRISPRi_Pi_rep2','Pi',min_val=0,max_val=4, log_transform=True)
# scp.pl.plotReplicateScatter(axes[2], adata, 'A549_CRISPRi_Pi_rep1','A549_CRISPRi_Pi_rep2','Pi',min_val=0,max_val=4, log_transform=True)

for ax in axes: 
    ax.tick_params(width=0.5)
    for i in ax.spines.values(): i.set_linewidth(0.5)
    # ax.spines['top'].set_visible(False)
    # ax.spines['right'].set_visible(False)
    # ax.spines['bottom'].set_visible(False)
    # ax.spines['left'].set_visible(False)
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label]):
        item.set_fontsize(8)
    for item in ax.get_xticklabels() + ax.get_yticklabels():
        item.set_fontsize(6)

# fig1.suptitle('PRDX1 parent screen',fontsize=12)

plt.show()

___

In [12]:
screenprocessing = scp.load.loadScreenProcessingData('../CRISPRi/Analysis/PiKiMiRiWi/PiKiMiRiWi_2reps')
screenprocessing['counts'].columns = 'A549_CRISPRi_' + screenprocessing['counts'].columns.to_frame().agg('_'.join, axis=1)

In [13]:
ol = list(set(screenprocessing['counts'].index) & set(adata.var.index))

In [14]:
len(ol)

102777

In [15]:
fig, axes = plt.subplots(1,4,figsize=(7,2.5))

axes[0].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,'A549_CRISPRi_T0_rep1']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,'A549_CRISPRi_T0_rep1'] + 1),
    s=.1, c='black', alpha=.2
)
axes[0].set_title('T0 Rep1')

axes[1].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,'A549_CRISPRi_T0_rep2']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,'A549_CRISPRi_T0_rep2'] + 1),
    s=.1, c='black', alpha=.2
)
axes[1].set_title('T0 Rep2')

axes[2].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,'A549_CRISPRi_DMSO_rep1']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,'A549_CRISPRi_DMSO_rep1'] + 1),
    s=.1, c='black', alpha=.2
)
axes[2].set_title('DMSO Rep1')

axes[3].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,'A549_CRISPRi_DMSO_rep2']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,'A549_CRISPRi_DMSO_rep2'] + 1),
    s=.1, c='black', alpha=.2
)
axes[3].set_title('DMSO Rep2')

for ax in axes:
    ax.set_xlabel('Screen Processing')
    ax.set_ylabel('ScreenPro2')
    ax.set_xlim([-0.1,4])
    ax.set_ylim([-0.1,4])
    
    ax.grid(None)
    
    ax.tick_params(width=0.5)
    for i in ax.spines.values(): i.set_linewidth(0.5)
    # ax.spines['top'].set_visible(False)
    # ax.spines['right'].set_visible(False)
    # ax.spines['bottom'].set_visible(False)
    # ax.spines['left'].set_visible(False)
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label]):
        item.set_fontsize(8)
    for item in ax.get_xticklabels() + ax.get_yticklabels():
        item.set_fontsize(6)
    
plt.suptitle('DDRi V2 Screens', fontsize=12)
plt.tight_layout()
plt.show()

In [16]:
fig, axes = plt.subplots(1,4,figsize=(7,2.5))

cond1 = "PiRi"
axes[0].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,f'A549_CRISPRi_{cond1}_rep1']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,f'A549_CRISPRi_{cond1}_rep1'] + 1),
    s=.1, c='black', alpha=.2
)
axes[0].set_title(f'{cond1} Rep1')

axes[1].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,'A549_CRISPRi_Wi_rep2']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,'A549_CRISPRi_Wi_rep2'] + 1),
    s=.1, c='black', alpha=.2
)
axes[1].set_title(f'{cond1} Rep2')

axes[2].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,'A549_CRISPRi_DMSO_rep1']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,'A549_CRISPRi_DMSO_rep1'] + 1),
    s=.1, c='black', alpha=.2
)
axes[2].set_title('DMSO Rep1')

axes[3].scatter(
    x = np.log10(screenprocessing['counts'].loc[ol,'A549_CRISPRi_DMSO_rep2']+ 1),
    y = np.log10(adata.to_df().T.loc[ol,'A549_CRISPRi_DMSO_rep2'] + 1),
    s=.1, c='black', alpha=.2
)
axes[3].set_title('DMSO Rep2')

for ax in axes:
    ax.set_xlabel('Screen Processing')
    ax.set_ylabel('ScreenPro2')
    ax.set_xlim([-0.1,4])
    ax.set_ylim([-0.1,4])
    
    ax.grid(None)
    
    ax.tick_params(width=0.5)
    for i in ax.spines.values(): i.set_linewidth(0.5)
    # ax.spines['top'].set_visible(False)
    # ax.spines['right'].set_visible(False)
    # ax.spines['bottom'].set_visible(False)
    # ax.spines['left'].set_visible(False)
    for item in ([ax.title, ax.xaxis.label, ax.yaxis.label]):
        item.set_fontsize(8)
    for item in ax.get_xticklabels() + ax.get_yticklabels():
        item.set_fontsize(6)
    
plt.suptitle('DDRi V2 Screens', fontsize=12)
plt.tight_layout()
plt.show()

___

In [17]:
adata.to_df().T.sum()[screenprocessing['counts'].columns] / 10**6

A549_CRISPRi_DMSO_rep1     48.718589
A549_CRISPRi_DMSO_rep2     28.086767
A549_CRISPRi_Ki_rep1       37.695912
A549_CRISPRi_Ki_rep2       23.865067
A549_CRISPRi_Mi_rep1       32.910419
A549_CRISPRi_Mi_rep2       46.517719
A549_CRISPRi_Pi_rep1       45.441083
A549_CRISPRi_Pi_rep2       33.736242
A549_CRISPRi_PiKi_rep1     37.236937
A549_CRISPRi_PiKi_rep2     78.800999
A549_CRISPRi_PiMi_rep1     37.714290
A549_CRISPRi_PiMi_rep2    108.729121
A549_CRISPRi_PiRi_rep1     43.599360
A549_CRISPRi_PiRi_rep2     19.454159
A549_CRISPRi_PiWi_rep1     40.225769
A549_CRISPRi_PiWi_rep2     37.929204
A549_CRISPRi_Ri_rep1       54.166237
A549_CRISPRi_Ri_rep2       32.538540
A549_CRISPRi_T0_rep1       48.168472
A549_CRISPRi_T0_rep2       33.978428
A549_CRISPRi_Wi_rep1       86.448619
A549_CRISPRi_Wi_rep2       34.393649
dtype: float64

In [18]:
screenprocessing['counts'].sum() / 10**6

A549_CRISPRi_DMSO_rep1     47.806961
A549_CRISPRi_DMSO_rep2     61.991303
A549_CRISPRi_Ki_rep1       37.715724
A549_CRISPRi_Ki_rep2       23.626238
A549_CRISPRi_Mi_rep1       32.924088
A549_CRISPRi_Mi_rep2       46.082196
A549_CRISPRi_Pi_rep1       44.641146
A549_CRISPRi_Pi_rep2       33.761415
A549_CRISPRi_PiKi_rep1     37.283167
A549_CRISPRi_PiKi_rep2     78.040945
A549_CRISPRi_PiMi_rep1     37.749345
A549_CRISPRi_PiMi_rep2    107.661539
A549_CRISPRi_PiRi_rep1     42.805998
A549_CRISPRi_PiRi_rep2     47.896952
A549_CRISPRi_PiWi_rep1     40.189403
A549_CRISPRi_PiWi_rep2     37.958185
A549_CRISPRi_Ri_rep1       53.273955
A549_CRISPRi_Ri_rep2       67.916446
A549_CRISPRi_T0_rep1       47.400609
A549_CRISPRi_T0_rep2       47.393014
A549_CRISPRi_Wi_rep1       85.908385
A549_CRISPRi_Wi_rep2       34.419751
dtype: float64

- replace older counts with the new counts for sanity checks.

In [19]:
adata_screenprocessing = ad.AnnData(
    X = screenprocessing['counts'].T,
)

adata_screenprocessing.obs = adata.obs.loc[adata_screenprocessing.obs.index,:]

adata_screenprocessing.var = library_table.set_index('sgId').loc[adata_screenprocessing.var.index]

adata_screenprocessing.var.rename(columns={'gene':'target','transcripts':'transcript'},inplace=True)

adata_screenprocessing.var['targetType'] = [
    'gene' if t != 'negative_control' else 'negative_control' for t in 
    adata_screenprocessing.var.target
]

# Step 2: Phenotype calculation

- [ ] 

### run phenoscore

In [24]:
screen = scp.PooledScreens(adata, verbose=True, n_reps=2)
# screen = scp.PooledScreens(adata_screenprocessing, verbose=True, n_reps=2)

screen.filterLowCounts()

screen.countNormalization()

104515 variables with less than 1 reads (filter_type: 'all')
Pseudocount added to counts.
Counts normalized by sequencing depth.


In [25]:
screen.calculateDrugScreen(
    score_level='compare_guides',
    untreated='DMSO', treated=['Pi','Ri','PiRi','Mi','PiMi','Wi','PiWi','Ki','PiKi'],
    t0='T0', 
    # count_filter_type='either',
    keep_top_n = 3,
    var_names=['target','transcript'],
    collapse_var='target'
    # run_name='compare_guides_top_3',
)

/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	DMSO vs T0


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	Pi vs T0
	Pi vs DMSO


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


...

	PiWi vs T0
	PiWi vs DMSO


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	Ki vs T0
	Ki vs DMSO


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	PiKi vs T0
	PiKi vs DMSO


___

In [26]:
screen.calculateDrugScreen(
    score_level='compare_reps',
    untreated='DMSO', treated=['Pi','Ri','PiRi','Mi','PiMi','Wi','PiWi','Ki','PiKi'],
    t0='T0', 
    # count_filter_type='either',
    # keep_top_n = 3,
    # run_name='compare_guides_top_3',
    var_names=['target','transcript'],
)

/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	DMSO vs T0


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	Pi vs T0
	Pi vs DMSO


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


...

	PiWi vs T0
	PiWi vs DMSO


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	Ki vs T0
	Ki vs DMSO


/home/abea/miniconda3/envs/screenpro2/lib/python3.9/site-packages/screenpro/assays/__init__.py:85: UserWarning: No doubling rate information provided.
  warnings.warn('No doubling rate information provided.')


	PiKi vs T0
	PiKi vs DMSO


___

In [27]:
scp.load._write_screen_pkl(screen,'A549_CRISPRi_screens')

Object successfully saved to "A549_CRISPRi_screens.pkl"


#### PRDX1 rank!

Luke:
> can you tell us the gene rank for PRDX1 for each screen using old analysis and new analysis

In [28]:
# old_analysis_hits = pd.read_csv(
#     '../results/Ci_all_gene_scores_with_hit_labels.csv',header=[0,1],index_col=0
# )

# [
#     (score_tag, old_analysis_hits[score_tag]['score'].rank()['PRDX1'].astype(int))
#     for score_tag in old_analysis_hits.columns.to_frame()[0].unique()
# ]

# [
#     (score_tag,
#      screen.phenotypes['compare_guides']['results'][score_tag]['score'].rank()['PRDX1'].astype(int))
#     for score_tag in screen.listPhenotypeScores(run_name='compare_guides')
#     if 'DMSO' in score_tag
# ]   

### extract result tables

In [9]:
screen = scp.load._read_screen_pkl('A549_CRISPRi_screens')

In [10]:
result_tables = []

for phenotype_name in screen.listPhenotypeScores(run_name='compare_guides'):
    if 'rho' in phenotype_name:
        result_tables.append((phenotype_name,
            screen.getPhenotypeScores(
                run_name='compare_guides', phenotype_name=phenotype_name,
                pvalue_col = 'ttest pvalue',
                threshold=6
            ).query('target!="negative_control"').set_index('target')
        ))
    
result_tables = dict(result_tables)

### get result tables

In [34]:
def getAnnotatedTables(screen, threshold):
    return dict([
        (phenotype_name, 
         screen.getPhenotypeScores(
             run_name='compare_guides',phenotype_name=phenotype_name,threshold=threshold, pvalue_col='ttest pvalue'
         ).query('target!="negative_control"').set_index('target'))
        for phenotype_name in screen.listPhenotypeScores(run_name='compare_guides')
    ])

In [37]:
annotated_result_table = pd.concat(dict([
    (drug_names[k.split(':')[1].split('_vs_')[0]],table) for k, table in 
    getAnnotatedTables(screen, threshold=6).items()
    if 'rho' in k
]),axis=1).dropna()

### save to file

In [185]:
with pd.ExcelWriter('A549_CRISPRi_screen_analysis.xlsx', engine='openpyxl') as writer:
    screen.adata.obs.to_excel(writer, sheet_name='sample sheet')
    screen.adata.to_df(layer='raw_counts').astype(int).T.to_excel(writer, sheet_name='raw counts')
    screen.adata.to_df(layer='seq_depth_norm').astype(int).T.to_excel(writer, sheet_name='normalized counts')
    getAnnotatedTable(screen, threshold=6).to_excel(writer, sheet_name='A549 CRISPRi screens')

# 

In [192]:
%reload_ext watermark

In [193]:
%watermark

Last updated: 2024-08-13T19:58:52.281703-07:00

Python implementation: CPython
Python version       : 3.9.19
IPython version      : 8.18.1

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 5.15.0-117-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 64
Architecture: 64bit



In [194]:
%watermark --iversions

numpy     : 1.26.4
matplotlib: 3.6.3
screenpro : 0.4.11
scanpy    : 1.9.6
upsetplot : 0.9.0
pandas    : 2.1.4
anndata   : 0.10.7



___